# Corriendo Varias Trayectorias HYSPLIT en ARGOS para diferentes tiempos y sitios

Se presentan los pasos para ejecutar HYSPLIT empleando la interfase de Jupyter para trayectorias.

1. Crear y ubicarse en el directorio de trabajo
1. Crear el archivo ASCDATA.CFG 
1. Configurar los parametros de la ejecución de trayectoria
1. Crear el archivo CONTROL
1. Ejecutar el programa de trayectorias
1. Crear el despliegue

In [6]:
%%bash -l
# en la variable DIR definir el nombre del directorio
DIR=enero
# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d ${HOME}/data/$DIR ];then mkdir $HOME/data/$DIR ;fi

# se ubica en el directorio
cd ${HOME}/data/${DIR}
#-------------------------------------------------------------
# set default directory structure if not passed through

  MDL="${HOME}/hysplit.v5.2.0_CentOS7.9.2009"
  OUT="${HOME}/data/${DIR}"
  MET="${HOME}/hysplit_data"
  cd $OUT

  if [ ! -f ASCDATA.CFG ]; then
     echo "-90.0  -180.0"     >ASCDATA.CFG 
     echo "0.1     0.1"      >>ASCDATA.CFG
     echo "1800   3600"      >>ASCDATA.CFG
     echo "2"                >>ASCDATA.CFG
     echo "0.2"              >>ASCDATA.CFG
     echo "'$MDL/bdyfiles/bdyfiles0p1/'" >>ASCDATA.CFG
  fi
  echo "### $0 ###"


#----------------------------------------------------------
# Configura archivo de control para  simulaciones de 
#  dispersion/concentracion 

iyr=19  # Anio solo los dos ultimos dígitos
ims=01  # Mes solo dos digitos
idy=15  # Dia de inicio de modelacion
ihr=06  # Hora de inicio de modelacion

#  Ubicación de Tuxpan a modelar y altura
lat=21.014   
lon=-97.334
lvl=600.0
#  Ubicación de Petacalco a modelar y altura
lat2=17.984  
lon2=-102.115
lvl2=700.0

#  
vop=0         # Opcion de movimiento vertical 
run=12        # Numero de horas a modelar
ztop=10000.0   # Altura del tope de modelacion

nmf=1         # Numero de archivos meteorologicos 
MET="${HOME}/hysplit_data" # Directorio donde estan los data meteorologicos
data="AR_${ims}${idy}20${iyr}.bin"  #nombre del archivo meteorologico

#--------------------------------------------------------------
#   Indica cada cuantas horas va a correr
stph=01
#   Indica el numero de dias a correr
idia=00
#   Indica el numero de horas despues de iniciar.
hfin=30

#--------------------------------------------------------------
#  calcula las horas totales
if [ $idia -lt 1 ] ; then
   ht=$(($hfin+10#${ihr}))
   else
    ht=$((10#${ihr}+${idia}*24+10#${hfin}))
   fi
#
# dias en el mes
#    D   E  F  M  A  M  J  J  A  S  O  N  D          
mes=(31 31 28 31 30 31 30 31 31 30 31 30 31)

#--------------------------------------------------------------
# Datos de tiempo en loop

for ih in $( seq $ihr $stph $ht )
do
 if [ $ih -gt 23 ] ; then
   lh=$(($ih -24*$(($ih/24))))
   ld=$((${idy}+ $ih/24 ))
   if [ $ld -lt 10 ]; then ld=0$ld ;fi
  else
   lh=$ih
   ld=$idy
   lm=$ims
 fi
 if [ $ld -gt ${mes[$ims]} ]; then
   lm=$(($ims + 1))
   ld=$((${idy}+ $ih/24 - ${mes[$ims]}))
   if [ $lm -lt 10 ]; then lm=0$lm ;fi
   if [ $ld -lt 10 ]; then ld=0$ld ;fi
 fi
if [ $lh -lt 10 ]; then lh=0$lh ;fi


# Creacion del archivo CONTROL
#----------------------------------------------------------
# 

  echo "$iyr $lm $ld $lh       " >CONTROL
  echo "2                      ">>CONTROL
  echo "$lat $lon $lvl         ">>CONTROL
  echo "$lat2 $lon2 $lvl2      ">>CONTROL
  echo "$run                   ">>CONTROL
  echo "$vop                   ">>CONTROL
  echo "$ztop                  ">>CONTROL
  echo "$nmf                   ">>CONTROL
  echo "$MET/                  ">>CONTROL
  echo "$data                  ">>CONTROL
  echo "$OUT/                  ">>CONTROL
  echo "tdump$iyr$lm$ld$lh     ">>CONTROL
 
#----------------------------------------------------------
# corre la simulacion

  rm -f SETUP.CFG
    
  ${MDL}/exec/hyts_std >sal
    echo "Terminado $iyr$lm$ld$lh"
done 


### bash ###
Terminado 19011506
Terminado 19011507
Terminado 19011508
Terminado 19011509
Terminado 19011510
Terminado 19011511
Terminado 19011512
Terminado 19011513
Terminado 19011514
Terminado 19011515
Terminado 19011516
Terminado 19011517
Terminado 19011518
Terminado 19011519
Terminado 19011520
Terminado 19011521
Terminado 19011522
Terminado 19011523
Terminado 19011600
Terminado 19011601
Terminado 19011602
Terminado 19011603
Terminado 19011604
Terminado 19011605
Terminado 19011606
Terminado 19011607
Terminado 19011608
Terminado 19011609
Terminado 19011610
Terminado 19011611
Terminado 19011612


In [ ]:
%%bash -l

ls  ../data/enero/tdump* |wc -l
tail -1 ../data/enero/tdump19010212

## Visualización de las trayectorias


In [9]:
%%bash -l
# en la variable DIR definir el nombre del directorio
DIR=enero
MDL="${HOME}/hysplit.v5.2.0_CentOS7.9.2009"


# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d ${HOME}/data/$DIR ];then mkdir $HOME/data/$DIR ;fi

# se ubica en el directorio
cd ${HOME}/data/${DIR}
# Genera el archivo de todas las trayectorias
#ls -r tdump1901011* > INFILE
ls -r tdump1901* > INFILE


 WEB=""
  if [ -f /bin/convert ];then WEB="/bin/convert";fi

echo "'$MDL/graphics/mexico.shp' 0 0.01 0.4 0.6 0.8" >shapefiles_mex.txt

#-------------------------------------------------------------
# set default directory structure if not passed through


echo "'TITLE&','Trayectorias de Enero 01 06 &'"  >LABELS.CFG
${MDL}/exec/trajplot  +g0 -i+INFILE -jshapefiles_mex.txt -l3 -v3 -otray.ps -z0  

${WEB} tray.ps tray.jpg
  

 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Processing: tdump19012512                           
 Processing: tdump19012511                           
 Processing: tdump19012510                           
 Processing: tdump19012509                           
 Processing: tdump19012508                           
 Processing: tdump19012507                           
 Processing: tdump19012506                           
 Processing: tdump19012505                           
 Processing: tdump19012504                           
 Processing: tdump19012503                           
 Processing: tdump19012502                           
 Processing: tdump19012501                           
 Processing: tdump19012500                           
 Processing: tdump19012423                           
 Processing: tdump19012422                           
 Processing: tdump19012421                           
 Processing: tdump19012420                  